# Install libs

In [ ]:
%pip install pytorch_fid torchmetrics scikit-image lpips piq  torch-fidelity piqa

# Import Libs

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter, ImageEnhance
import cv2  # [[1]][[2]]
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import csv
import datetime
from piqa import SSIM  # << Install with `pip install piqa` [[9]]
import torch_fidelity
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import lpips
import torch
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import piqa
from skimage.util import random_noise  # << Install first [[1]][[2]]
import skimage.util  # << Add this import at the top of your file [[1]]
import pathlib
import pandas as pd
from pathlib import Path
from sklearn.metrics.pairwise import euclidean_distances

# Variables for ECC GAN

In [ ]:
bs_c = '/content/drive/.shortcut-targets-by-id/11uft3Rv8T4FYMqiUyk7cTxJ7DDDjnn79/Classification/'
dt_c = [bs_c+'Train/',bs_c+'Test/']

bs = '/content/drive/.shortcut-targets-by-id/1UGQF2LHCrsyEmb1WB-Afz_en4bRRYQWv/Segmentation/'
dt = [bs+'Original/',bs+'Ground_Truth/',bs+'Ground_Truth_New/',bs+'Original_New/']
pths=['Basophil','Eosinophil','Lymphocyte','Monocyte','Neutrophil']
# Faghat roo Basophil javab dade
aim = 0 # 'Basophil 0 ok','Eosinophil 1 ok','Lymphocyte 2 ok az 10-30','Monocyte 3 400 epoch az 300 be bad','Neutrophil 4 30 va 40 khoobe baghie ok nist'
aim_pths = [d + pths[aim] + '/' for d in dt]  # << Corrected line [[1]][[2]]
aim_pths_c = [d + pths[aim] + '/' for d in dt_c]  # << Corrected line [[1]][[2]]

a_dt_pth = aim_pths_c
# a_dt_pth = aim_pths[0]
# a_dt_pth = aim_pths[-1]

# b_dt_pth = aim_pths[1]
# b_dt_pth = aim_pths[2] # Ground Truth New
b_dt_pth = aim_pths[-1]


root = '/content/drive/MyDrive/Colab Notebooks/WBC_DB/'
res_c = root + 'Cy/' + pths[aim] + '/'
res_mdl_c = root + 'Cy/model/' + pths[aim] + '/'
lss_c = root + 'Cy/loss/' + pths[aim] + '/'

res_g = root + 'G/' + pths[aim] + '/'
res_mdl_g = root + 'G/model/' + pths[aim] + '/'
lss_g = root + 'G/loss/' + pths[aim] + '/'

a_ge_pth_c = root + 'GeCy/' + pths[aim] + '_A' # دسته A
b_ge_pth_c = root + 'GeCy/' + pths[aim] + '_B' # دسته B

ge_pth_g = root + 'GeG/' + pths[aim]

os.makedirs(res_c, exist_ok=True)
os.makedirs(res_mdl_c, exist_ok=True)
os.makedirs(lss_c, exist_ok=True)

os.makedirs(res_g, exist_ok=True)
os.makedirs(res_mdl_g, exist_ok=True)
os.makedirs(lss_g, exist_ok=True)

os.makedirs(a_ge_pth_c, exist_ok=True)
os.makedirs(b_ge_pth_c, exist_ok=True)

os.makedirs(ge_pth_g, exist_ok=True)

# a_dt_imgs = [os.path.join(a_dt_pth, f) for f in os.listdir(a_dt_pth) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
a_dt_imgs = []
for path in a_dt_pth:
  a_dt_imgs.extend([os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
random.shuffle(a_dt_imgs)
a_dt_imgs = a_dt_imgs[:300]  # << Your random subset [[1]][[3]] 795  = len Monocyte

b_dt_imgs = [os.path.join(b_dt_pth, f) for f in os.listdir(b_dt_pth) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
dt_imgs_raw = a_dt_imgs + b_dt_imgs

a_ge_imgs_c = [os.path.join(a_ge_pth_c, f) for f in os.listdir(a_ge_pth_c) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
b_ge_imgs_c = [os.path.join(b_ge_pth_c, f) for f in os.listdir(b_ge_pth_c) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
dt_imgs_c = a_ge_imgs_c + b_ge_imgs_c


# dt_imgs = dt_imgs_raw + dt_imgs_c
dt_imgs = dt_imgs_c + a_dt_imgs


ge_imgs_g = [os.path.join(ge_pth_g, f) for f in os.listdir(ge_pth_g) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

print(f"A data for {pths[aim]}:",len(a_dt_imgs),a_dt_imgs)
print(f"B data for {pths[aim]}:",len(b_dt_imgs),b_dt_imgs)

print(f"Cycle Generated A for {pths[aim]}:",len(a_ge_imgs_c),a_ge_imgs_c)
print(f"Cycle Generated B for {pths[aim]}:",len(b_ge_imgs_c),b_ge_imgs_c)
print(f"Cycle Generated B for {pths[aim]}:",len(b_ge_imgs_c),b_ge_imgs_c)
print(f"All Cycle generated for {pths[aim]}:",len(dt_imgs_c),dt_imgs_c)
print(f"All Raw data for {pths[aim]}:",len(dt_imgs_raw),dt_imgs_raw)
print(f"All data for {pths[aim]}:",len(dt_imgs),dt_imgs)
print(f"All GAN Generated  {pths[aim]}:",len(ge_imgs_g),ge_imgs_g)

# Variables for R3GAN

In [ ]:
num_or = 1000 #1000
num_a =  300 #300
num_b = 700 #700
bs_c = '/content/drive/.shortcut-targets-by-id/11uft3Rv8T4FYMqiUyk7cTxJ7DDDjnn79/Classification/'
dt_c = [bs_c+'Train/',bs_c+'Test/']

pths=['Basophil','Eosinophil','Lymphocyte','Monocyte','Neutrophil']
# Faghat roo Basophil javab dade
aim = 4 # 'Basophil 0 ok','Eosinophil 1 ok','Lymphocyte 2 ok az 10-30','Monocyte 3 400 epoch az 300 be bad','Neutrophil 4 30 va 40 khoobe baghie ok nist'
aim_pths_c = [d + pths[aim] + '/' for d in dt_c]  # << Corrected line [[1]][[2]]

or_dt_pth = aim_pths_c



root = '/content/drive/MyDrive/Colab Notebooks/WBC_DB/'



res_g = root + 'G/' + pths[aim] + '/'
res_mdl_g = root + 'G/model/' + pths[aim] + '/'
lss_g = root + 'G/loss/' + pths[aim] + '/'

a_ge_pth_c = root + 'GeCy/' + pths[aim] + '_A' # دسته A
b_ge_pth_c = root + 'GeCy/' + pths[aim] + '_B' # دسته B


ge_pth_g = root + 'GeG/' + pths[aim]

os.makedirs(root, exist_ok=True)
os.makedirs(res_g, exist_ok=True)
os.makedirs(res_mdl_g, exist_ok=True)
os.makedirs(lss_g, exist_ok=True)

os.makedirs(a_ge_pth_c, exist_ok=True)
os.makedirs(b_ge_pth_c, exist_ok=True)

os.makedirs(ge_pth_g, exist_ok=True)



or_imgs = []
for path in or_dt_pth:
  or_imgs.extend([os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
random.shuffle(or_imgs)
or_imgs = or_imgs[:num_or]  # << Your random subset [[1]][[3]] 795  = len Monocyte

a_ge_imgs_c = [os.path.join(a_ge_pth_c, f) for f in os.listdir(a_ge_pth_c) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
random.shuffle(a_ge_imgs_c)
a_ge_imgs_c = a_ge_imgs_c[:num_a]  # << Your random subset [[1]][[3]] 795  = len Monocyte

b_ge_imgs_c = [os.path.join(b_ge_pth_c, f) for f in os.listdir(b_ge_pth_c) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
random.shuffle(b_ge_imgs_c)
b_ge_imgs_c = b_ge_imgs_c[:num_b]  # << Your random subset [[1]][[3]] 795  = len Monocyte

dt_imgs = or_imgs + a_ge_imgs_c + b_ge_imgs_c


ge_imgs_g = [os.path.join(ge_pth_g, f) for f in os.listdir(ge_pth_g) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

dt_prpsd = or_imgs + ge_imgs_g


print(f"Orginal Data {pths[aim]}:",len(or_imgs),or_imgs)

print(f"Cycle Data A {pths[aim]}:",len(a_ge_imgs_c),a_ge_imgs_c)
print(f"Cycle Data B {pths[aim]}:",len(b_ge_imgs_c),b_ge_imgs_c)

print(f"All GAN DATA  {pths[aim]}:",len(dt_imgs),dt_imgs)

print(f"All GAN Generated  {pths[aim]}:",len(ge_imgs_g),ge_imgs_g)

print(f"Data Proposed  {pths[aim]}:",len(dt_prpsd),dt_prpsd)


# GEG+RAW=PROPOSED-DT -->> Basophil: 1100+301=1401, Eosinophil: 595+1000=1595, Lymphocyte: 975+1000=1975, Monocyte: 670+795=1465, Neutrophil: 850+1000=1850

## Train on CPU for temperory testing

In [ ]:

# results_path = res_g


# تنظیمات اولیه
latent_dim = 4 #128
image_size = 256  # اندازه تصویر 256x256 256
batch_size = 1   # 8 # کاهش batch size برای تصاویر بزرگتر
num_epochs = 5   # افزایش تعداد ایپوک برای آموزش بهتر

intrvl = 2

# Create dataset from paths
train_dataset = tf.data.Dataset.from_tensor_slices(dt_imgs)

# Define preprocessing function (ensure it returns a tensor with known shape)
def load_and_preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, dtype=tf.float32)  # << Read as RGB [[1]]
    img.set_shape([image_size, image_size, 3])  # << Fixed shape (256x256x3) [[2]]
    img = tf.image.resize(img, [image_size, image_size])  # << Resize ensures consistent dimensions [[2]]
    img = (img / 127.5) - 1.0  # << Normalize to [-1, 1] [[5]]
    return img

train_dataset = train_dataset.map(
    load_and_preprocess_image,
    num_parallel_calls=tf.data.AUTOTUNE
).batch(batch_size).shuffle(buffer_size=1000)  # << Final dataset setup [[1]][[2]]

def build_generator(latent_dim, image_size):
    model = tf.keras.Sequential()
    init_size = image_size // 32
    model.add(layers.Dense(init_size * init_size * 512, use_bias=False, input_shape=(latent_dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((init_size, init_size, 512)))

    model.add(layers.Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(32, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    return model

# تعریف تشخیص‌گر (Discriminator) برای تصاویر 256x256
def build_discriminator(image_size):
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=[image_size, image_size, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(512, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model



# Define generator and discriminator
generator = build_generator(latent_dim, image_size)
discriminator = build_discriminator(image_size)

# تابع از دست رساندن
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# تعریف تابع از دست رساندن برای مولد
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# تعریف تابع از دست رساندن برای تشخیص‌گر
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# بهینه‌سازی
generator_optimizer = tf.keras.optimizers.Adam(1e-5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5)

# Training loop (ensure shapes are correct before training)
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# آموزش مدل
def train(dataset, epochs):
    # ایجاد فایل CSV اگر وجود ندارد
    csv_file = lss_g + pths[-1] + '_cygan_loss.csv'
    if not os.path.exists(csv_file):
        with open(csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Epoch", "Generator Loss", "Discriminator Loss"])

    for epoch in range(epochs):
      for image_batch in dataset:
          gen_loss, disc_loss = train_step(image_batch)

      with open(csv_file, mode='a', newline='') as file:
          writer = csv.writer(file)
          writer.writerow([epoch + 1, gen_loss.numpy(), disc_loss.numpy()])
      print(f"Epoch {epoch}/{epochs}, Generator Loss: {gen_loss.numpy():.4f}, Discriminator Loss: {disc_loss.numpy():.4f}")
      generate_and_save_images(generator, epoch + 1, seed,sv=False)
      # هر 20 تا در پنجره پایین نمایش میده
      # if epoch % intrvl == 0:
      #       # نمایش و ذخیره تصاویر تولید شده
      #       generate_and_save_images(generator, epoch + 1, seed,sv=True)

# تابع نمایش و ذخیره تصاویر رنگی
def generate_and_save_images(model, epoch, test_input,sv=False):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(8, 8), dpi = 72)
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        img = (predictions[i] * 127.5 + 127.5).numpy().astype(np.uint8)
        plt.imshow(img)
        plt.axis('off')
    plt.show()
    if sv:
      save_path = os.path.join(res_g, f'epc_{epoch:04d}.png')
      plt.savefig(save_path)
    plt.close(fig)  # بستن شکل برای جلوگیری از مصرف حافظه زیاد

# تعریف ورودی ثابت برای نمایش تصاویر
seed = tf.random.normal([16, latent_dim])

# شروع آموزش
# train(train_dataset, num_epochs)

In [ ]:
# Train the model
train(train_dataset, num_epochs)

## Train on GPU

In [ ]:


# === Initial Setup ===
image_size = 256
latent_dim = 100  # Fixed latent dimension [[1]]
batch_size = 2
num_epochs = 40
intrvl = 5
lr = 0.00005 #0.0001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

csv_file = Path(lss_g + pths[aim] + '_g_lss.csv')
csv_file.touch(exist_ok=True)

# === Dataset Creation ===
class ImageDataset(Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths
        self.transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1,1] [[5]]
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        img = Image.open(path).convert('RGB')
        return self.transform(img)

train_dataset = ImageDataset(dt_imgs)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Verify dataset shape
for batch in train_loader:
    assert batch.shape == (batch_size, 3, image_size, image_size), f"Batch shape mismatch: {batch.shape}"
    break

# === Generator Architecture ===
class Generator(nn.Module):
    def __init__(self, latent_dim, image_size):
        super().__init__()
        self.latent_dim = latent_dim
        self.image_size = image_size

        # Initial linear projection
        self.initial_linear = nn.Linear(latent_dim, 512 * 8 * 8)  # (batch, 512*8*8)

        # Reshape to (batch, 512, 8, 8)
        self.up1 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        )  # → 16x16

        self.up2 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        )  # → 32x32

        self.up3 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True)
        )  # → 64x64

        self.up4 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=True)
        )  # → 128x128

        self.final = nn.Sequential(
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )  # → 256x256

    def forward(self, x):
        # Map noise to linear space
        x = self.initial_linear(x)
        # Reshape to 512x8x8 [[1]]
        x = x.view(-1, 512, 8, 8)  # << Critical reshape [[1]]

        x = self.up1(x)
        x = self.up2(x)
        x = self.up3(x)
        x = self.up4(x)
        return self.final(x)

# === Discriminator Architecture ===
class Discriminator(nn.Module):
    def __init__(self, image_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.AdaptiveAvgPool2d(1),  # Global pooling [[5]]
            nn.Flatten(),  # (batch, 512)
            nn.Linear(512, 1)  # Scalar output [[5]]
        )

    def forward(self, x):
        return self.model(x)

# Initialize models
generator = Generator(latent_dim, image_size).to(device)
discriminator = Discriminator(image_size).to(device)

# Define loss and optimizers
criterion = nn.BCEWithLogitsLoss().to(device)  # Binary cross entropy [[5]]

generator_optimizer = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Custom weight initialization
def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
        torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)  # << Replicating TF initialization [[3]]

generator.apply(weights_init)
discriminator.apply(weights_init)

# === Loss Functions ===
def generator_loss(fake_output):
    return criterion(fake_output, torch.ones_like(fake_output).to(device))

def discriminator_loss(real_output, fake_output):
    real_loss = criterion(real_output, torch.ones_like(real_output).to(device))
    fake_loss = criterion(fake_output, torch.zeros_like(fake_output).to(device))
    return real_loss + fake_loss

# === Training Loop ===
def train_step(images):
    valid = torch.ones(images.size(0), 1, device=device)
    fake = torch.zeros(images.size(0), 1, device=device)

    # Train Generator
    generator_optimizer.zero_grad()
    z = torch.randn(images.size(0), latent_dim, device=device)
    generated_images = generator(z)
    g_loss = generator_loss(discriminator(generated_images))
    g_loss.backward()
    generator_optimizer.step()

    # Train Discriminator
    discriminator_optimizer.zero_grad()
    real_pred = discriminator(images)
    fake_pred = discriminator(generated_images.detach())
    d_loss_real = criterion(real_pred, valid)
    d_loss_fake = criterion(fake_pred, fake)
    d_loss = d_loss_real + d_loss_fake
    d_loss.backward()
    discriminator_optimizer.step()

    return g_loss.item(), d_loss.item()

# === Image Generation Function ===
# def generate_and_save_images(model, epoch, test_input, sv=False):
#     model.eval()
#     with torch.no_grad():
#         predictions = model(test_input).cpu()
#     if sv:
#       d=300
#     else:
#       d=36
#     fig = plt.figure(figsize=(8, 8), dpi=d)
#     for i in range(predictions.shape[0]):
#         plt.subplot(4, 4, i+1)
#         img = ((predictions[i].numpy().transpose(1, 2, 0) + 1) / 2 * 255).astype(np.uint8)
#         plt.imshow(img)
#         plt.axis('off')

#     if sv:
#         plt.savefig(os.path.join(res_g, f'epc_{epoch:04d}.png'))
#     plt.show()
#     plt.close(fig)


def generate_and_save_images(model, epoch, sv=False):

    model.eval()  # Set model to evaluation mode
    num_samples=16
    # Generate new random noise each time
    noise = torch.randn(num_samples, latent_dim, device=device)  # << Random seed every call [[1]]

    with torch.no_grad():
        generated_images = model(noise).cpu()  # Generate and move to CPU [[2]]

    d = 36
    if sv:
      d = 300
    # Prepare to show images
    fig = plt.figure(figsize=(8, 8), dpi=d)
    rows = cols = int(np.ceil(np.sqrt(num_samples)))  # Dynamic layout

    for i in range(num_samples):
        plt.subplot(rows, cols, i+1)
        plt.axis('off')
        img = ((generated_images[i] * 0.5 + 0.5) * 255).clamp(0, 255).permute(1, 2, 0).numpy().astype(np.uint8)
        plt.imshow(img)
        if sv:
          plt.savefig(os.path.join(res_g, f'epc_{epoch:04d}.png'),bbox_inches='tight')

    plt.tight_layout()
    plt.show()
    plt.close(fig)
# === Start Training ===
def train(dataset, epochs):
    flg = True
    with open(csv_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Epoch", "Generator Loss", "Discriminator Loss"])

    for epoch in range(epochs):
        for real_images in dataset:
            real_images = real_images.to(device)
            gen_loss, disc_loss = train_step(real_images)
        with open(csv_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([epoch+1, gen_loss, disc_loss])

        if (epoch + 1) % intrvl == 0:
            flg = False
            generate_and_save_images(generator, epoch+1, sv=True)
            torch.save(generator.state_dict(), f"{res_mdl_g}e_{epoch+1}.pth")

        else:
            flg = True
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"  Generator Loss: {gen_loss:.4f}")
        print(f"  Discriminator Loss: {disc_loss:.4f}")
        if flg:
          generate_and_save_images(generator, epoch+1, sv=False)

# Initialize seed
# seed = torch.randn(16, latent_dim, device=device)

In [ ]:
# Start training
print(f"Using {'GPU' if device.type == 'cuda' else 'CPU'} acceleration")
print(pths[aim])
print('batch_size: ', batch_size)
print('num_epochs: ', num_epochs)
print('lr: ', lr)
print('image_size: ', image_size)
print('intrvl: ', intrvl)
train(train_loader, num_epochs)

## load model in order to show and generate new images from GANs

In [ ]:
def generate_timestamp_decimal():
    current_time = datetime.datetime.now()  # [[2]]
    date_part = current_time.strftime("%d-%m-%y")  # Day-Month-Last two digits of year [[3]]
    time_part = current_time.strftime("%H-%M-%S")  # Hour:Minute:Second [[2]]
    decimal_num = int(round(random.uniform(0, 1) * 1000))  # << Key adjustment [[5]][[9]]
    return f"{date_part}-{time_part}-{decimal_num}"

result = generate_timestamp_decimal()
print(result)  # Output: "15-04-25 14:30:45 0.123" [[1]][[2]][[3]]

In [ ]:
# New method for generating random images from saved model
def generate_random_images(model_path, num_images=16, sv=False,sh=False,o_d=None,ep=0):

    # Load model architecture
    gen = Generator(latent_dim,image_size).to(device)

    # Load weights
    try:
        gen.load_state_dict(torch.load(model_path, map_location=device))
        print(f"Model loaded from {model_path}")
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    gen.eval()

    # Generate random noise
    noise = torch.randn(num_images, latent_dim, device=device)

    # Generate images
    with torch.no_grad():
        fake_images = gen(noise)

    # Create figure for visualization
    if sh:
      fig = plt.figure(figsize=(8, 8))
      columns = min(4, num_images)
      rows = (num_images + columns - 1) // columns

      for i in range(num_images):
          plt.subplot(rows, columns, i+1)
          img = ((fake_images[i].cpu() * 0.5 + 0.5) * 255).clamp(0, 255).permute(1, 2, 0).numpy().astype(np.uint8)
          plt.imshow(img)
          plt.axis('off')

      plt.tight_layout()
      plt.show()
      plt.close(fig)


    # Save individual images if output_dir specified
    if sv:
      for i in range(num_images):
          img = ((fake_images[i].cpu() * 0.5 + 0.5) * 255).clamp(0, 255).permute(1, 2, 0).numpy().astype(np.uint8)
          Image.fromarray(img).save(f"{o_d}/geg_epc_{str(ep)}_{i:03d}_{generate_timestamp_decimal()}.png")

In [ ]:
e = 70 # 10 20 30 40 50 60 70
latest_model = f"{res_mdl_g}e_{e}.pth"
# generate_random_images(latest_model, num_images=20, sv=False, sh=True, o_d=ge_pth_g,ep=e)
# generate_random_images(latest_model, num_images=250, sv=True, sh=False, o_d=ge_pth_g,ep=e)

# ECC GAN implementation

## Device Varibles

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# image_size = 256  # ابعاد تصویر
# batch_size = 1
# num_epochs = 20
# # dim = 2048
# intrvl = 4 # میگیم هر چند اپوک مد رو ذخیره کنه
# lr = 0.0001 #0.0001
# lambda_cycle = 10.0 #10
# lambda_identity = 0.5 #0.5

image_size = 256  # ابعاد تصویر
intrvl = 4 # میگیم هر چند اپوک مد رو ذخیره کنه
batch_size = 8 #بالاتر از ۱۶ نمیشه
num_epochs = 16
lr = 0.0001 #0.0001
lambda_cycle = 10 #10
lambda_identity = 0.25 #0.5

# if aim == 0:
#   batch_size = 8
#   num_epochs = 100
#   lambda_cycle = 20 #10
#   lambda_identity = 0.125 #0.5
#   lr = 0.0001 #0.0001
#   lambda_cycle = 10.0 #10
#   lambda_identity = 0.5 #0.5

print(pths[aim])

## Transformers

In [ ]:
# تعریف تبدیل تصویر
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # نرمالایز به [-1,1] [[5]]
])

# کلاس داده سیت برای پیش‌پردازش تصاویر
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, paths_A, paths_B):
        self.paths_A = paths_A
        self.paths_B = paths_B
        self.transform = transform

    def __getitem__(self, index):
        img_A = Image.open(self.paths_A[index % len(self.paths_A)]).convert("RGB")
        img_B = Image.open(self.paths_B[index % len(self.paths_B)]).convert("RGB")
        return self.transform(img_A), self.transform(img_B)

    def __len__(self):
        return max(len(self.paths_A), len(self.paths_B))

# ساخت داده سیت
def create_dataset(paths_A, paths_B):
    dataset = ImageDataset(paths_A, paths_B)
    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )

# Residual Block برای مولد
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        return out + identity  # [[7]]



class Generator(nn.Module):
    def __init__(self,
                 init_filters=64,  # لایه اول Downsampling [[1]]
                 mid_filters=128,  # لایه دوم Downsampling [[1]]
                 res_filters=256,  # لایه Residual [[1]]
                 num_residuals=9  # تعداد Blockهای Residual [[2]]
                ):
        super(Generator, self).__init__()  # << استفاده صحیح از super() [[3]][[4]]

        # Downsampling
        self.down1 = nn.Sequential(
            nn.Conv2d(3, init_filters, kernel_size=7, padding=3),
            nn.ReLU(inplace=True)
        )
        self.down2 = nn.Sequential(
            nn.Conv2d(init_filters, mid_filters, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(mid_filters),
            nn.ReLU(inplace=True)
        )
        self.down3 = nn.Sequential(
            nn.Conv2d(mid_filters, res_filters, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(res_filters),
            nn.ReLU(inplace=True)
        )

        # Residual Blocks
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(res_filters) for _ in range(num_residuals)]
        )

        # Upsampling
        self.up1 = nn.Sequential(
            nn.ConvTranspose2d(res_filters, mid_filters, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(mid_filters),
            nn.ReLU(inplace=True)
        )
        self.up2 = nn.Sequential(
            nn.ConvTranspose2d(mid_filters, init_filters, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(init_filters),
            nn.ReLU(inplace=True)
        )
        self.final = nn.Sequential(
            nn.Conv2d(init_filters, 3, kernel_size=7, padding=3),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.down1(x)
        x = self.down2(x)
        x = self.down3(x)
        x = self.res_blocks(x)
        x = self.up1(x)
        x = self.up2(x)
        return self.final(x)


# مولد ها
generator_AtoB = Generator(
    init_filters=32,    # کاهش اولین لایه Downsampling [[1]]
    mid_filters=64,     # کاهش لایه دوم Downsampling [[1]]
    res_filters=128,    # کاهش تعداد فیلترهای Residual [[1]]
    num_residuals=6     # کاهش تعداد Blockهای Residual [[2]]
)
generator_BtoA = Generator(
    init_filters=32,
    mid_filters=64,
    res_filters=128,
    num_residuals=6
)

class Discriminator(nn.Module):
    def __init__(self,
                 in_channels=3,
                 filters=[64, 128, 256],  # لایههای کانولوشنی [[1]]
                 kernel_size=4,
                 stride=2
                ):
        super(Discriminator, self).__init__()

        layers = []
        current_filters = in_channels
        for filt in filters:
            layers.append(
                nn.Conv2d(current_filters, filt, kernel_size, stride, padding=1)
            )
            if filt != filters[-1]:  # برای آخرین لایه BatchNorm نیاز نیست
                layers.append(nn.BatchNorm2d(filt))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            current_filters = filt

        # لایه آخر
        layers.append(nn.Conv2d(current_filters, 1, kernel_size, stride=1, padding=1))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# تشخیصگرها
discriminator_A = Discriminator(
    filters=[64, 128, 128]  # کاهش تعداد فیلترهای لایه سوم [[1]]
)
discriminator_B = Discriminator(
    filters=[64, 128, 128]
)


# تابع هزینه
criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()




# بهینه‌سازی
optimizer_G = optim.Adam(
    list(generator_AtoB.parameters()) + list(generator_BtoA.parameters()),
    lr=lr,
    betas=(0.5, 0.999)
)
optimizer_D_A = optim.Adam(discriminator_A.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D_B = optim.Adam(discriminator_B.parameters(), lr=lr, betas=(0.5, 0.999))  # << اصلاح اینجا

# تابع نمایش
def generate_and_save_images(epoch, test_A, test_B):
    generator_AtoB.eval()
    generator_BtoA.eval()
    # Random indices within batch sizes [[7]][[10]]
    batch_size_A = test_A.size(0)
    batch_size_B = test_B.size(0)
    idx_A = random.randint(0, batch_size_A - 1)
    idx_B = random.randint(0, batch_size_B - 1)
    # real_A_name = os.path.basename(a_dt_imgs[idx_A])
    # real_B_name = os.path.basename(b_dt_imgs[idx_B])

    with torch.no_grad():
        fake_B = generator_AtoB(test_A)
        fake_A = generator_BtoA(test_B)
        recon_A = generator_BtoA(fake_B)
        recon_B = generator_AtoB(fake_A)
    fntsz = 36
    fntwt = 40

    fig = plt.figure(figsize=(15, 8))
    plt.subplot(2,3,1)
    plt.imshow((test_A[idx_A].cpu().numpy().transpose(1,2,0)+1)/2)
    # plt.title(real_A_name, fontsize=fntsz, fontweight=fntwt)
    plt.title("Real A", fontsize=fntsz, fontweight=fntwt)
    plt.axis('off')


    plt.subplot(2,3,2)
    plt.imshow((fake_B[idx_A].cpu().numpy().transpose(1,2,0)+1)/2)
    plt.title("Fake B", fontsize=fntsz, fontweight=fntwt)
    plt.axis('off')


    plt.subplot(2,3,3)
    plt.imshow((recon_A[idx_A].cpu().numpy().transpose(1,2,0)+1)/2)
    plt.title("Reconstructed A", fontsize=fntsz, fontweight=fntwt)
    plt.axis('off')


    plt.subplot(2,3,4)
    plt.imshow((test_B[idx_B].cpu().numpy().transpose(1,2,0)+1)/2)
    # plt.title(real_B_name, fontsize=fntsz, fontweight=fntwt)
    plt.title("Real B", fontsize=fntsz, fontweight=fntwt)

    plt.axis('off')


    plt.subplot(2,3,5)
    plt.imshow((fake_A[idx_B].cpu().numpy().transpose(1,2,0)+1)/2)
    plt.title("Fake A", fontsize=fntsz, fontweight=fntwt)
    plt.axis('off')


    plt.subplot(2,3,6)
    plt.imshow((recon_B[idx_B].cpu().numpy().transpose(1,2,0)+1)/2)
    plt.title("Reconstructed B", fontsize=fntsz, fontweight=fntwt)
    plt.axis('off')

    plt.savefig(f"{res_c}/e_{epoch}.png")
    plt.show()
    plt.close()

# حلقه آموزش
def train(dataset, epochs):

    generator_AtoB.to(device)
    generator_BtoA.to(device)
    discriminator_A.to(device)
    discriminator_B.to(device)
    # Before training starts (initialize CSV):
    csv_file = lss_c + pths[-1] + '_cygan_loss.csv'
    with open(csv_file, 'w', newline='') as f:
      writer = csv.writer(f)
      writer.writerow(['Epoch', 'Generator Loss', 'Discriminator A Loss', 'Discriminator B Loss'])  # [[2]][[3]]

    for epoch in range(epochs):
        for i, (real_A, real_B) in enumerate(dataset):
            real_A = real_A.to(device)
            real_B = real_B.to(device)

            # آموزش مولد
            optimizer_G.zero_grad()

            # تولید تصاویر
            fake_B = generator_AtoB(real_A)
            fake_A = generator_BtoA(real_B)

            # بازسازی
            recon_A = generator_BtoA(fake_B)
            recon_B = generator_AtoB(fake_A)

            # هذلولی
            identity_A = generator_BtoA(real_A)
            identity_B = generator_AtoB(real_B)

            # Loss های GAN
            loss_GAN_A = criterion_GAN(discriminator_B(fake_B), torch.ones_like(discriminator_B(fake_B)))  # [[5]]
            loss_GAN_B = criterion_GAN(discriminator_A(fake_A), torch.ones_like(discriminator_A(fake_A)))

            # Loss سیکل
            loss_cycle = criterion_cycle(recon_A, real_A) + criterion_cycle(recon_B, real_B)

            # Loss هذلولی
            loss_identity = criterion_identity(identity_A, real_A) + criterion_identity(identity_B, real_B)

            # Loss مجموع
            total_loss_G = loss_GAN_A + loss_GAN_B + lambda_cycle * loss_cycle + lambda_identity * loss_identity
            total_loss_G.backward()
            optimizer_G.step()

            # آموزش تشخیصگر A
            optimizer_D_A.zero_grad()
            pred_real_A = discriminator_A(real_A)
            pred_fake_A = discriminator_A(fake_A.detach())

            loss_D_A = criterion_GAN(pred_real_A, torch.ones_like(pred_real_A)) + \
                      criterion_GAN(pred_fake_A, torch.zeros_like(pred_fake_A))
            loss_D_A.backward()
            optimizer_D_A.step()

            # آموزش تشخیصگر B
            optimizer_D_B.zero_grad()
            pred_real_B = discriminator_B(real_B)
            pred_fake_B = discriminator_B(fake_B.detach())

            loss_D_B = criterion_GAN(pred_real_B, torch.ones_like(pred_real_B)) + \
                      criterion_GAN(pred_fake_B, torch.zeros_like(pred_fake_B))
            loss_D_B.backward()
            optimizer_D_B.step()

        print(f"Epoch {epoch+1}/{epochs}:")
        print(f"  Generator Loss: {total_loss_G.item():.4f}")
        print(f"  Discriminator A Loss: {loss_D_A.item():.4f} | Discriminator B Loss: {loss_D_B.item():.4f}")
        # save css
        with open(csv_file, 'a', newline='') as f:
          writer = csv.writer(f)
          writer.writerow([
          epoch+1,
          f"{total_loss_G.item():.4f}",
          f"{loss_D_A.item():.4f}",
          f"{loss_D_B.item():.4f}"
        ])

        if (epoch + 1) % intrvl == 0:
            torch.save(generator_AtoB.state_dict(), f"{res_mdl_c}/generator_AtoB_epoch_{epoch+1}.pth")
            torch.save(generator_BtoA.state_dict(), f"{res_mdl_c}/generator_BtoA_epoch_{epoch+1}.pth")
        # تصاویر نمونه برای نمایش غیر تکراری
        random_batch = next(iter(dataset))
        test_A = random_batch[0].to(device)
        test_B = random_batch[1].to(device)

        generate_and_save_images(epoch+1, test_A, test_B)

root = '/content/drive/MyDrive/Colab Notebooks/WBC_DB/'
res_c = root + 'Cy/' + pths[aim] + '/'
res_mdl_c = root + 'Cy/model/' + pths[aim] + '/'
lss_c = root + 'Cy/loss/' + pths[aim] + '/'

## Start Train

In [ ]:
dataset = create_dataset(a_dt_imgs, b_dt_imgs)
print(pths[aim])
print('batch_size: ', batch_size)
print('num_epochs: ', num_epochs)
print('lambda_cycle: ', lambda_cycle)
print('lambda_identity: ', lambda_identity)
print('lr: ', lr)
print('image_size: ', image_size)
print('intrvl: ', intrvl)

# train(dataset, num_epochs)

## naming new images

In [ ]:
def generate_timestamp_decimal():
    current_time = datetime.datetime.now()  # [[2]]
    date_part = current_time.strftime("%d-%m-%y")  # Day-Month-Last two digits of year [[3]]
    time_part = current_time.strftime("%H-%M-%S")  # Hour:Minute:Second [[2]]
    decimal_num = int(round(random.uniform(0, 1) * 1000))  # << Key adjustment [[5]][[9]]


    return f"{date_part}-{time_part}-{decimal_num}"

result = generate_timestamp_decimal()
print(result)  # Output: "15-04-25 14:30:45 0.123" [[1]][[2]][[3]]

In [ ]:
cnt_g_img = 50 # 4 8 12 16 ...100
mdl_ep = 100 # 4 8 12 16 ...100
print(pths[aim])

## Generate new images

In [ ]:
def generate_show_save_images(impths, generator, test_images, epoch, output_dir='', save=False,filter_bw=True):
    generator.eval()
    with torch.no_grad():
        fake_images = generator(test_images.to(device))

    valid_images = []
    for img_tensor in fake_images:
        img = ((img_tensor.cpu().numpy().transpose(1,2,0) + 1) * 127.5).astype(np.uint8)

        # Filter logic
        black_mask = np.all(img < 20, axis=-1)
        black_percent = (np.sum(black_mask) / img.size) * 100
        gray_mask = np.var(img, axis=-1) < 30
        gray_percent = (np.sum(gray_mask) / img.size) * 100
        if filter_bw:
          if black_percent <= 10 and gray_percent <= 10:
              valid_images.append((img, black_percent, gray_percent))
        else:
              valid_images.append((img, black_percent, gray_percent))


    # Early exit if no valid images
    if not valid_images:
        print("No valid images to display.")
        return

    # Calculate layout
    cols = int(np.ceil(np.sqrt(len(valid_images))))
    rows = int(np.ceil(len(valid_images) / cols))

    # Create figure with correct dimensions
    fig = plt.figure(figsize=(cols*2, rows*2),dpi=300)
    b = 0
    # Plot valid images
    for i, (img, _, _) in enumerate(valid_images):
        plt.subplot(rows, cols, i+1)
        plt.imshow(img)
        filename = impths[b].split('/')[-1]  # Use os.path.basename(img_path) for cross-platform
        plt.title(f"{i}  {filename}", fontsize=4)
        b = b+1
        plt.axis('off')
        if save:
          Image.fromarray(img).save(os.path.join(output_dir, f"{i}_epoch_{epoch}_{generate_timestamp_decimal()}.png"))  # << ذخیره سازی [[7]]


    # Save and show
    # plt.savefig(os.path.join(output_dir, f"filtered_epoch_{epoch}.png"))
    plt.show()
    plt.close()

In [ ]:
generator_AtoB = Generator(
    init_filters=32,    # کاهش اولین لایه Downsampling [[1]]
    mid_filters=64,     # کاهش لایه دوم Downsampling [[1]]
    res_filters=128,    # کاهش تعداد فیلترهای Residual [[1]]
    num_residuals=6     # کاهش تعداد Blockهای Residual [[2]]
)
generator_AtoB.load_state_dict(torch.load(res_mdl+"generator_AtoB_epoch_"+ str(mdl_ep)+".pth",map_location=device))
generator_AtoB.to(device).eval()
# random.shuffle(a_imgs)
# test_A_paths = a_imgs[:cnt_g_img]

random.shuffle(a_dt_imgs)
test_A_paths = a_dt_imgs[:cnt_g_img]
test_A = []
for path in test_A_paths:
    img = Image.open(path).convert("RGB")
    img = transform(img).unsqueeze(0).to(device)
    test_A.append(img)
test_A = torch.cat(test_A, dim=0)  # [[9]]

# ساخت تصاویر
generate_show_save_images(
    impths=test_A_paths,
    generator=generator_AtoB,
    test_images=test_A,
    epoch=mdl_ep,
    output_dir=a_ge_pth,
    filter_bw=False
    # save=True
)


## reconstrcture images


In [ ]:
def generate_show_save_images(
    impths,
    generator_a2b,
    generator_b2a,
    test_images,
    epoch,
    output_dir_a='',
    output_dir_b='',
    save=False,
    filter_bw=True,
    show=False
):
    def gr(fake):
        # Calculate layout: 2 columns (fake and recon), rows based on valid count

        saturation_threshold = 127  # Adjust this to define "gray" (lower = more tolerance) [[1]]
        min_value = 10  #50           # Mid-intensity range (50-200) [[2]]
        max_value = 60 #200
        pil_img = Image.fromarray(fake)
        hsv_img = pil_img.convert("HSV")
        hsv_array = np.array(hsv_img)
        # === Brighten Gray Pixels ===
        gray_mask = hsv_array[..., 1] < saturation_threshold  # Low saturation (gray pixels)
        mid_value_mask = (hsv_array[..., 2] > min_value) & (hsv_array[..., 2] < max_value)
        combined_mask = gray_mask & mid_value_mask
        # Increase Value (brightness) for gray pixels in midtones
        hsv_array[..., 2][combined_mask] += 50  # << Adjust this value (e.g., 50 → 100 for stronger brightening) [[3]]
        hsv_array[..., 2] = np.clip(hsv_array[..., 2], 0, 255)  # Clamp values to valid range
        # Convert back to RGB and save
        enhanced_hsv = Image.fromarray(hsv_array, "HSV")
        enhanced_rgb = enhanced_hsv.convert("RGB")
        return enhanced_rgb



    def re(recon): # با افزایش کانتراست و ...و چرخش
        pil_recon = Image.fromarray(recon)
        # === Random Rotation (Avoid Black Borders) ===
        angle = random.choice([90, 180, 270])
        rotated = pil_recon.rotate(angle, expand=True)  # << Use expand=True [[1]]
        # === Boost Brightness (Random Factor 1.0–2.0) ===
        brightness_enhancer = ImageEnhance.Brightness(rotated)
        brightness_factor = random.uniform(1.0, 2.0)  # << Increase brightness [[2]]
        brightened = brightness_enhancer.enhance(brightness_factor)
        # === Enhance Contrast (Random Factor 1.0–2.0) ===
        contrast_enhancer = ImageEnhance.Contrast(brightened)
        contrast_factor = random.uniform(1.0, 2.0)  # << Sharpen contrast [[3]]
        enhanced = contrast_enhancer.enhance(contrast_factor)
        return enhanced

    generator_a2b.eval()
    generator_b2a.eval()
    with torch.no_grad():
        # Generate fake images
        fake_images = generator_a2b(test_images.to(device))
        # Generate reconstructed images (B→A)
        recon_images = generator_b2a(fake_images)  # << Reconstruction added [[1]]

    valid_images = []
    for fake_tensor, recon_tensor, img_path in zip(fake_images, recon_images, impths):
        # Process fake image
        fake = ((fake_tensor.cpu().numpy().transpose(1, 2, 0) + 1) * 127.5).astype(np.uint8)
        # Process reconstructed image
        recon = ((recon_tensor.cpu().numpy().transpose(1, 2, 0) + 1) * 127.5).astype(np.uint8)

        # Filter fake images (reconstructed images are not filtered)
        black_mask = np.all(fake < 20, axis=-1)
        black_percent = (np.sum(black_mask) / fake.size) * 100
        gray_mask = np.var(fake, axis=-1) < 30
        gray_percent = (np.sum(gray_mask) / fake.size) * 100
        if filter_bw:
            if black_percent <= 10 and gray_percent <= 10:
                valid_images.append((fake, recon, img_path))  # << Include recon [[2]]
        else:
            valid_images.append((fake, recon, img_path))

    # Early exit if no valid images
    if not valid_images:
        print("No valid fake images to display.")
        return
    num_pairs = len(valid_images)
    rows = num_pairs
    cols = 2
    # Create figure

    for i, (fake, recon, img_path) in enumerate(valid_images):
        # === Fake Image ===
        filename = os.path.basename(img_path)
        if show:
          fig = plt.figure(figsize=(cols * 4, rows * 4), dpi=72)  # << Adjusted size for 2 columns [[3]]
          plt.subplot(rows, cols, 2*i + 1)  # << First column (fake)
          plt.title(f"Fake {i} - {filename}", fontsize=8)
          plt.axis('off')
          # plt.imshow(fake)
          plt.imshow(gr(fake))
          plt.subplot(rows, cols, 2*i + 2)  # << Second column (reconstructed)
          plt.title(f"Recon {i} - {filename}", fontsize=8)
          plt.axis('off')
          # plt.imshow(recon)
          plt.imshow(re(recon))
          plt.show()
          plt.tight_layout()
          plt.close()
        if save:
            fake_path = os.path.join(output_dir_a, f"fk_{i}_e_{epoch}_{generate_timestamp_decimal()}.png")
            # Image.fromarray(fake).save(fake_path)
            gr(fake).save(fake_path)  # << Save the modified image [[4]]
            recon_path = os.path.join(output_dir_b, f"rcn_{i}_e_{epoch}_{generate_timestamp_decimal()}.png")
            # Image.fromarray(recon).save(recon_path)
            # Convert array to PIL Image
            re(recon).save(recon_path)


    # Final adjustments
    # if save:
    #   pass
    #     # plt.savefig(os.path.join(output_dir, f"epoch_{epoch}_comparison.png"))  # << Save comparison figure [[4]]


In [ ]:
cnt_g_img = 50 # 4 8 12 16 ...100
mdl_ep = 100 # 4 8 12 16 ...100
print(pths[aim])
generator_BtoA = Generator(
    init_filters=32,    # کاهش اولین لایه Downsampling [[1]]
    mid_filters=64,     # کاهش لایه دوم Downsampling [[1]]
    res_filters=128,    # کاهش تعداد فیلترهای Residual [[1]]
    num_residuals=6     # کاهش تعداد Blockهای Residual [[2]]
)

generator_BtoA.load_state_dict(torch.load(res_mdl+"generator_BtoA_epoch_"+ str(mdl_ep)+".pth",map_location=device))
generator_BtoA.to(device).eval()

generator_AtoB = Generator(
    init_filters=32,    # کاهش اولین لایه Downsampling [[1]]
    mid_filters=64,     # کاهش لایه دوم Downsampling [[1]]
    res_filters=128,    # کاهش تعداد فیلترهای Residual [[1]]
    num_residuals=6     # کاهش تعداد Blockهای Residual [[2]]
)
generator_AtoB.load_state_dict(torch.load(res_mdl+"generator_AtoB_epoch_"+ str(mdl_ep)+".pth",map_location=device))
generator_AtoB.to(device).eval()


# random.shuffle(a_dt_imgs)
# test_A_paths = a_dt_imgs[:cnt_g_img]
test_A_paths = a_dt_imgs
test_A = []
for path in test_A_paths:
    img = Image.open(path).convert("RGB")
    img = transform(img).unsqueeze(0).to(device)
    test_A.append(img)
test_A = torch.cat(test_A, dim=0)  # [[9]]



# During training, pass both generators and test images from domain A
generate_show_save_images(
    impths=test_A_paths,  # << Original paths of test images [[5]]
    generator_a2b=generator_AtoB,
    generator_b2a=generator_BtoA,
    test_images=test_A,  # << Test images from domain A [[5]]
    epoch=mdl_ep,
    output_dir_a=a_ge_pth,
    output_dir_b=b_ge_pth,
    save=True,
    filter_bw=False,
    show=False
)

Basophil


## bw images

In [ ]:
def generate_show_save_images(
    impths,
    generator_a2b,
    generator_b2a,
    test_images,
    epoch,
    output_dir_a='',
    output_dir_b='',
    save=False,
    filter_bw=True,
    show=False
):
    def gr(fake):
        # Calculate layout: 2 columns (fake and recon), rows based on valid count

        saturation_threshold = 127  # Adjust this to define "gray" (lower = more tolerance) [[1]]
        min_value = 10  #50           # Mid-intensity range (50-200) [[2]]
        max_value = 60 #200
        pil_img = Image.fromarray(fake)
        hsv_img = pil_img.convert("HSV")
        hsv_array = np.array(hsv_img)
        # === Brighten Gray Pixels ===
        gray_mask = hsv_array[..., 1] < saturation_threshold  # Low saturation (gray pixels)
        mid_value_mask = (hsv_array[..., 2] > min_value) & (hsv_array[..., 2] < max_value)
        combined_mask = gray_mask & mid_value_mask
        # Increase Value (brightness) for gray pixels in midtones
        hsv_array[..., 2][combined_mask] += 50  # << Adjust this value (e.g., 50 → 100 for stronger brightening) [[3]]
        hsv_array[..., 2] = np.clip(hsv_array[..., 2], 0, 255)  # Clamp values to valid range
        # Convert back to RGB and save
        enhanced_hsv = Image.fromarray(hsv_array, "HSV")
        enhanced_rgb = enhanced_hsv.convert("RGB")
        return enhanced_rgb



    def re(recon): # با افزایش کانتراست و ...و چرخش
        pil_recon = Image.fromarray(recon)
        # === Random Rotation (Avoid Black Borders) ===
        angle = random.choice([90, 180, 270])
        rotated = pil_recon.rotate(angle, expand=True)  # << Use expand=True [[1]]
        # === Boost Brightness (Random Factor 1.0–2.0) ===
        brightness_enhancer = ImageEnhance.Brightness(rotated)
        brightness_factor = random.uniform(1.0, 2.0)  # << Increase brightness [[2]]
        brightened = brightness_enhancer.enhance(brightness_factor)
        # === Enhance Contrast (Random Factor 1.0–2.0) ===
        contrast_enhancer = ImageEnhance.Contrast(brightened)
        contrast_factor = random.uniform(1.0, 2.0)  # << Sharpen contrast [[3]]
        enhanced = contrast_enhancer.enhance(contrast_factor)
        return enhanced

    generator_a2b.eval()
    generator_b2a.eval()
    with torch.no_grad():
        # Generate fake images
        fake_images = generator_a2b(test_images.to(device))
        # Generate reconstructed images (B→A)
        recon_images = generator_b2a(fake_images)  # << Reconstruction added [[1]]

    valid_images = []
    for fake_tensor, recon_tensor, img_path in zip(fake_images, recon_images, impths):
        # Process fake image
        fake = ((fake_tensor.cpu().numpy().transpose(1, 2, 0) + 1) * 127.5).astype(np.uint8)
        # Process reconstructed image
        recon = ((recon_tensor.cpu().numpy().transpose(1, 2, 0) + 1) * 127.5).astype(np.uint8)

        # Filter fake images (reconstructed images are not filtered)
        black_mask = np.all(fake < 20, axis=-1)
        black_percent = (np.sum(black_mask) / fake.size) * 100
        gray_mask = np.var(fake, axis=-1) < 30
        gray_percent = (np.sum(gray_mask) / fake.size) * 100
        if filter_bw:
            if black_percent <= 10 and gray_percent <= 10:
                valid_images.append((fake, recon, img_path))  # << Include recon [[2]]
        else:
            valid_images.append((fake, recon, img_path))

    # Early exit if no valid images
    if not valid_images:
        print("No valid fake images to display.")
        return
    num_pairs = len(valid_images)
    rows = num_pairs
    cols = 2
    # Create figure

    for i, (fake, recon, img_path) in enumerate(valid_images):
        # === Fake Image ===
        filename = os.path.basename(img_path)
        if show:
          fig = plt.figure(figsize=(cols * 4, rows * 4), dpi=72)  # << Adjusted size for 2 columns [[3]]
          plt.subplot(rows, cols, 2*i + 1)  # << First column (fake)
          plt.title(f"Fake {i} - {filename}", fontsize=8)
          plt.axis('off')
          plt.imshow(fake)
          # plt.imshow(gr(fake))
          plt.subplot(rows, cols, 2*i + 2)  # << Second column (reconstructed)
          plt.title(f"Recon {i} - {filename}", fontsize=8)
          plt.axis('off')
          # plt.imshow(recon)
          plt.imshow(re(recon))
          plt.show()
          plt.tight_layout()
          plt.close()
        if save:
            fake_path = os.path.join(output_dir_a, f"fk_{i}_e_{epoch}_{generate_timestamp_decimal()}.png")
            # Image.fromarray(fake).save(fake_path)
            # gr(fake).save(fake_path)  # << Save the modified image [[4]]
            re(fake).save(fake_path)
            recon_path = os.path.join(output_dir_b, f"rcn_{i}_e_{epoch}_{generate_timestamp_decimal()}.png")
            # Image.fromarray(recon).save(recon_path)
            # Convert array to PIL Image
            re(recon).save(recon_path)


    # Final adjustments
    # if save:
    #   pass
    #     # plt.savefig(os.path.join(output_dir, f"epoch_{epoch}_comparison.png"))  # << Save comparison figure [[4]]


In [ ]:
# cnt_g_img = 2 # 4 8 12 16
mdl_ep = 16 # 4 8 12 16
print(f'epoch: {mdl_ep}')
print(pths[aim])
generator_BtoA = Generator(
    init_filters=32,    # کاهش اولین لایه Downsampling [[1]]
    mid_filters=64,     # کاهش لایه دوم Downsampling [[1]]
    res_filters=128,    # کاهش تعداد فیلترهای Residual [[1]]
    num_residuals=6     # کاهش تعداد Blockهای Residual [[2]]
)

generator_BtoA.load_state_dict(torch.load(res_mdl_c+"generator_BtoA_epoch_"+ str(mdl_ep)+".pth",map_location=device))
generator_BtoA.to(device).eval()

generator_AtoB = Generator(
    init_filters=32,    # کاهش اولین لایه Downsampling [[1]]
    mid_filters=64,     # کاهش لایه دوم Downsampling [[1]]
    res_filters=128,    # کاهش تعداد فیلترهای Residual [[1]]
    num_residuals=6     # کاهش تعداد Blockهای Residual [[2]]
)
generator_AtoB.load_state_dict(torch.load(res_mdl_c+"generator_AtoB_epoch_"+ str(mdl_ep)+".pth",map_location=device))
generator_AtoB.to(device).eval()


# random.shuffle(a_dt_imgs)
# test_A_paths = a_dt_imgs[:cnt_g_img]
a_dt_imgs = []
for path in a_dt_pth:
  a_dt_imgs.extend([os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
random.shuffle(a_dt_imgs)
a_dt_imgs = a_dt_imgs[:300]  # << Your random subset [[1]][[3]] 795  = len Monocyte
print(f"A data for {pths[aim]}:",len(a_dt_imgs),a_dt_imgs)


test_A_paths = a_dt_imgs
test_A = []
for path in test_A_paths:
    img = Image.open(path).convert("RGB")
    img = transform(img).unsqueeze(0).to(device)
    test_A.append(img)
test_A = torch.cat(test_A, dim=0)  # [[9]]



# During training, pass both generators and test images from domain A
generate_show_save_images(
    impths=test_A_paths,  # << Original paths of test images [[5]]
    generator_a2b=generator_AtoB,
    generator_b2a=generator_BtoA,
    test_images=test_A,  # << Test images from domain A [[5]]
    epoch=mdl_ep,
    output_dir_a=a_ge_pth_c,
    output_dir_b=b_ge_pth_c,
    save=True,
    filter_bw=False,
    show=False
)

epoch: 16
Neutrophil
A data for Neutrophil: 300 ['/content/drive/.shortcut-targets-by-id/11uft3Rv8T4FYMqiUyk7cTxJ7DDDjnn79/Classification/Test/Neutrophil/95-8-6-1_325_2.jpg', '/content/drive/.shortcut-targets-by-id/11uft3Rv8T4FYMqiUyk7cTxJ7DDDjnn79/Classification/Test/Neutrophil/95-8-6-1_255_2.jpg', '/content/drive/.shortcut-targets-by-id/11uft3Rv8T4FYMqiUyk7cTxJ7DDDjnn79/Classification/Train/Neutrophil/95-8-13-4_67_2.jpg', '/content/drive/.shortcut-targets-by-id/11uft3Rv8T4FYMqiUyk7cTxJ7DDDjnn79/Classification/Test/Neutrophil/95-8-22-3_115_1.jpg', '/content/drive/.shortcut-targets-by-id/11uft3Rv8T4FYMqiUyk7cTxJ7DDDjnn79/Classification/Train/Neutrophil/95-8-20-3_123_1.jpg', '/content/drive/.shortcut-targets-by-id/11uft3Rv8T4FYMqiUyk7cTxJ7DDDjnn79/Classification/Train/Neutrophil/95-8-10-1_262_1.jpg', '/content/drive/.shortcut-targets-by-id/11uft3Rv8T4FYMqiUyk7cTxJ7DDDjnn79/Classification/Train/Neutrophil/95-8-1-1_245_1.jpg', '/content/drive/.shortcut-targets-by-id/11uft3Rv8T4FYMqiUyk7cT

#R3GAN implementation

In [ ]:

# results_path = res_g


# تنظیمات اولیه
latent_dim = 4 #128
image_size = 256  # اندازه تصویر 256x256 256
batch_size = 1   # 8 # کاهش batch size برای تصاویر بزرگتر
num_epochs = 5   # افزایش تعداد ایپوک برای آموزش بهتر

intrvl = 2

# Create dataset from paths
train_dataset = tf.data.Dataset.from_tensor_slices(dt_imgs)

# Define preprocessing function (ensure it returns a tensor with known shape)
def load_and_preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, dtype=tf.float32)  # << Read as RGB [[1]]
    img.set_shape([image_size, image_size, 3])  # << Fixed shape (256x256x3) [[2]]
    img = tf.image.resize(img, [image_size, image_size])  # << Resize ensures consistent dimensions [[2]]
    img = (img / 127.5) - 1.0  # << Normalize to [-1, 1] [[5]]
    return img

train_dataset = train_dataset.map(
    load_and_preprocess_image,
    num_parallel_calls=tf.data.AUTOTUNE
).batch(batch_size).shuffle(buffer_size=1000)  # << Final dataset setup [[1]][[2]]

def build_generator(latent_dim, image_size):
    model = tf.keras.Sequential()
    init_size = image_size // 32
    model.add(layers.Dense(init_size * init_size * 512, use_bias=False, input_shape=(latent_dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((init_size, init_size, 512)))

    model.add(layers.Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(32, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    return model

# تعریف تشخیص‌گر (Discriminator) برای تصاویر 256x256
def build_discriminator(image_size):
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=[image_size, image_size, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(512, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model



# Define generator and discriminator
generator = build_generator(latent_dim, image_size)
discriminator = build_discriminator(image_size)

# تابع از دست رساندن
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# تعریف تابع از دست رساندن برای مولد
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# تعریف تابع از دست رساندن برای تشخیص‌گر
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# بهینه‌سازی
generator_optimizer = tf.keras.optimizers.Adam(1e-5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5)

# Training loop (ensure shapes are correct before training)
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# آموزش مدل
def train(dataset, epochs):
    # ایجاد فایل CSV اگر وجود ندارد
    csv_file = lss_g + pths[-1] + '_cygan_loss.csv'
    if not os.path.exists(csv_file):
        with open(csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Epoch", "Generator Loss", "Discriminator Loss"])

    for epoch in range(epochs):
      for image_batch in dataset:
          gen_loss, disc_loss = train_step(image_batch)

      with open(csv_file, mode='a', newline='') as file:
          writer = csv.writer(file)
          writer.writerow([epoch + 1, gen_loss.numpy(), disc_loss.numpy()])
      print(f"Epoch {epoch}/{epochs}, Generator Loss: {gen_loss.numpy():.4f}, Discriminator Loss: {disc_loss.numpy():.4f}")
      generate_and_save_images(generator, epoch + 1, seed,sv=False)
      # هر 20 تا در پنجره پایین نمایش میده
      # if epoch % intrvl == 0:
      #       # نمایش و ذخیره تصاویر تولید شده
      #       generate_and_save_images(generator, epoch + 1, seed,sv=True)

# تابع نمایش و ذخیره تصاویر رنگی
def generate_and_save_images(model, epoch, test_input,sv=False):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(8, 8), dpi = 72)
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        img = (predictions[i] * 127.5 + 127.5).numpy().astype(np.uint8)
        plt.imshow(img)
        plt.axis('off')
    plt.show()
    if sv:
      save_path = os.path.join(res_g, f'epc_{epoch:04d}.png')
      plt.savefig(save_path)
    plt.close(fig)  # بستن شکل برای جلوگیری از مصرف حافظه زیاد

# تعریف ورودی ثابت برای نمایش تصاویر
seed = tf.random.normal([16, latent_dim])

# شروع آموزش
# train(train_dataset, num_epochs)
train(train_dataset, num_epochs)

In [ ]:


# === Initial Setup ===
image_size = 256
latent_dim = 100  # Fixed latent dimension [[1]]
batch_size = 2
num_epochs = 40
intrvl = 5
lr = 0.00005 #0.0001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

csv_file = Path(lss_g + pths[aim] + '_g_lss.csv')
csv_file.touch(exist_ok=True)

# === Dataset Creation ===
class ImageDataset(Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths
        self.transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1,1] [[5]]
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        img = Image.open(path).convert('RGB')
        return self.transform(img)

train_dataset = ImageDataset(dt_imgs)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Verify dataset shape
for batch in train_loader:
    assert batch.shape == (batch_size, 3, image_size, image_size), f"Batch shape mismatch: {batch.shape}"
    break

# === Generator Architecture ===
class Generator(nn.Module):
    def __init__(self, latent_dim, image_size):
        super().__init__()
        self.latent_dim = latent_dim
        self.image_size = image_size

        # Initial linear projection
        self.initial_linear = nn.Linear(latent_dim, 512 * 8 * 8)  # (batch, 512*8*8)

        # Reshape to (batch, 512, 8, 8)
        self.up1 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        )  # → 16x16

        self.up2 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        )  # → 32x32

        self.up3 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True)
        )  # → 64x64

        self.up4 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=True)
        )  # → 128x128

        self.final = nn.Sequential(
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )  # → 256x256

    def forward(self, x):
        # Map noise to linear space
        x = self.initial_linear(x)
        # Reshape to 512x8x8 [[1]]
        x = x.view(-1, 512, 8, 8)  # << Critical reshape [[1]]

        x = self.up1(x)
        x = self.up2(x)
        x = self.up3(x)
        x = self.up4(x)
        return self.final(x)

# === Discriminator Architecture ===
class Discriminator(nn.Module):
    def __init__(self, image_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.AdaptiveAvgPool2d(1),  # Global pooling [[5]]
            nn.Flatten(),  # (batch, 512)
            nn.Linear(512, 1)  # Scalar output [[5]]
        )

    def forward(self, x):
        return self.model(x)

# Initialize models
generator = Generator(latent_dim, image_size).to(device)
discriminator = Discriminator(image_size).to(device)

# Define loss and optimizers
criterion = nn.BCEWithLogitsLoss().to(device)  # Binary cross entropy [[5]]

generator_optimizer = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Custom weight initialization
def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
        torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)  # << Replicating TF initialization [[3]]

generator.apply(weights_init)
discriminator.apply(weights_init)

# === Loss Functions ===
def generator_loss(fake_output):
    return criterion(fake_output, torch.ones_like(fake_output).to(device))

def discriminator_loss(real_output, fake_output):
    real_loss = criterion(real_output, torch.ones_like(real_output).to(device))
    fake_loss = criterion(fake_output, torch.zeros_like(fake_output).to(device))
    return real_loss + fake_loss

# === Training Loop ===
def train_step(images):
    valid = torch.ones(images.size(0), 1, device=device)
    fake = torch.zeros(images.size(0), 1, device=device)

    # Train Generator
    generator_optimizer.zero_grad()
    z = torch.randn(images.size(0), latent_dim, device=device)
    generated_images = generator(z)
    g_loss = generator_loss(discriminator(generated_images))
    g_loss.backward()
    generator_optimizer.step()

    # Train Discriminator
    discriminator_optimizer.zero_grad()
    real_pred = discriminator(images)
    fake_pred = discriminator(generated_images.detach())
    d_loss_real = criterion(real_pred, valid)
    d_loss_fake = criterion(fake_pred, fake)
    d_loss = d_loss_real + d_loss_fake
    d_loss.backward()
    discriminator_optimizer.step()

    return g_loss.item(), d_loss.item()

# === Image Generation Function ===
# def generate_and_save_images(model, epoch, test_input, sv=False):
#     model.eval()
#     with torch.no_grad():
#         predictions = model(test_input).cpu()
#     if sv:
#       d=300
#     else:
#       d=36
#     fig = plt.figure(figsize=(8, 8), dpi=d)
#     for i in range(predictions.shape[0]):
#         plt.subplot(4, 4, i+1)
#         img = ((predictions[i].numpy().transpose(1, 2, 0) + 1) / 2 * 255).astype(np.uint8)
#         plt.imshow(img)
#         plt.axis('off')

#     if sv:
#         plt.savefig(os.path.join(res_g, f'epc_{epoch:04d}.png'))
#     plt.show()
#     plt.close(fig)


def generate_and_save_images(model, epoch, sv=False):

    model.eval()  # Set model to evaluation mode
    num_samples=16
    # Generate new random noise each time
    noise = torch.randn(num_samples, latent_dim, device=device)  # << Random seed every call [[1]]

    with torch.no_grad():
        generated_images = model(noise).cpu()  # Generate and move to CPU [[2]]

    d = 36
    if sv:
      d = 300
    # Prepare to show images
    fig = plt.figure(figsize=(8, 8), dpi=d)
    rows = cols = int(np.ceil(np.sqrt(num_samples)))  # Dynamic layout

    for i in range(num_samples):
        plt.subplot(rows, cols, i+1)
        plt.axis('off')
        img = ((generated_images[i] * 0.5 + 0.5) * 255).clamp(0, 255).permute(1, 2, 0).numpy().astype(np.uint8)
        plt.imshow(img)
        if sv:
          plt.savefig(os.path.join(res_g, f'epc_{epoch:04d}.png'),bbox_inches='tight')

    plt.tight_layout()
    plt.show()
    plt.close(fig)
# === Start Training ===
def train(dataset, epochs):
    flg = True
    with open(csv_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Epoch", "Generator Loss", "Discriminator Loss"])

    for epoch in range(epochs):
        for real_images in dataset:
            real_images = real_images.to(device)
            gen_loss, disc_loss = train_step(real_images)
        with open(csv_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([epoch+1, gen_loss, disc_loss])

        if (epoch + 1) % intrvl == 0:
            flg = False
            generate_and_save_images(generator, epoch+1, sv=True)
            torch.save(generator.state_dict(), f"{res_mdl_g}e_{epoch+1}.pth")

        else:
            flg = True
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"  Generator Loss: {gen_loss:.4f}")
        print(f"  Discriminator Loss: {disc_loss:.4f}")
        if flg:
          generate_and_save_images(generator, epoch+1, sv=False)

# Initialize seed
# seed = torch.randn(16, latent_dim, device=device)
# Start training
print(f"Using {'GPU' if device.type == 'cuda' else 'CPU'} acceleration")
print(pths[aim])
print('batch_size: ', batch_size)
print('num_epochs: ', num_epochs)
print('lr: ', lr)
print('image_size: ', image_size)
print('intrvl: ', intrvl)
train(train_loader, num_epochs)

## load model to generate new images

In [ ]:
def generate_timestamp_decimal():
    current_time = datetime.datetime.now()  # [[2]]
    date_part = current_time.strftime("%d-%m-%y")  # Day-Month-Last two digits of year [[3]]
    time_part = current_time.strftime("%H-%M-%S")  # Hour:Minute:Second [[2]]
    decimal_num = int(round(random.uniform(0, 1) * 1000))  # << Key adjustment [[5]][[9]]
    return f"{date_part}-{time_part}-{decimal_num}"

result = generate_timestamp_decimal()
print(result)  # Output: "15-04-25 14:30:45 0.123" [[1]][[2]][[3]]
# New method for generating random images from saved model
def generate_random_images(model_path, num_images=16, sv=False,sh=False,o_d=None,ep=0):

    # Load model architecture
    gen = Generator(latent_dim,image_size).to(device)

    # Load weights
    try:
        gen.load_state_dict(torch.load(model_path, map_location=device))
        print(f"Model loaded from {model_path}")
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    gen.eval()

    # Generate random noise
    noise = torch.randn(num_images, latent_dim, device=device)

    # Generate images
    with torch.no_grad():
        fake_images = gen(noise)

    # Create figure for visualization
    if sh:
      fig = plt.figure(figsize=(8, 8))
      columns = min(4, num_images)
      rows = (num_images + columns - 1) // columns

      for i in range(num_images):
          plt.subplot(rows, columns, i+1)
          img = ((fake_images[i].cpu() * 0.5 + 0.5) * 255).clamp(0, 255).permute(1, 2, 0).numpy().astype(np.uint8)
          plt.imshow(img)
          plt.axis('off')

      plt.tight_layout()
      plt.show()
      plt.close(fig)


    # Save individual images if output_dir specified
    if sv:
      for i in range(num_images):
          img = ((fake_images[i].cpu() * 0.5 + 0.5) * 255).clamp(0, 255).permute(1, 2, 0).numpy().astype(np.uint8)
          Image.fromarray(img).save(f"{o_d}/geg_epc_{str(ep)}_{i:03d}_{generate_timestamp_decimal()}.png")



# GAN Mertics: Fréchet Inception Distance (FID) Inception Score (IS) Structural Similarity Index (SSIM) Perceptual Path Length (PPL) LPIPS (Learned Perceptual Image Patch Similarity) Mode Score Precision & Recall   Kernel Inception Distance (KID) PSNR (Peak Signal-to-Noise Ratio)  (Distribution Approximation (Probability Calibration (Distribution Matching  
'''
1- Fréchet Inception Distance (FID) #Ok
2- Inception Score (IS) #Ok
3- Kernel Inception Distance (KID) #Ok
4- LPIPS (Learned Perceptual Image Patch Similarity) #Ok
5- Structural Similarity Index (SSIM) # ok
6- PSNR (Peak Signal-to-Noise Ratio) # ok
7- Perceptual Path Length (PPL)
8- Mode Score Precision & Recall
9- (Distribution Approximation)
10- (Probability Calibration)
11- (Distribution Matching)

'''

In [ ]:
def evaluate_gan_metrics_1_4(original_dir, generated_dir):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Calculate FID, Inception Score, KID using torch_fidelity
    metrics = torch_fidelity.calculate_metrics(
        input1=original_dir,  # Original images directory [[1]]
        input2=generated_dir,  # Generated images directory [[1]]
        cuda=torch.cuda.is_available(),
        isc=True,  # Inception Score
        fid=True,  # Fréchet Inception Distance
        kid=True,  # Kernel Inception Distance
        kid_subset_size=10  # Adjust based on dataset size [[6]]
    )

    # Extract metrics
    fid_score = metrics['frechet_inception_distance']
    is_mean = metrics['inception_score_mean']
    kid_mean = metrics['kernel_inception_distance_mean']

    # SSIM and PSNR with proper resizing and win_size
    original_images = [
        os.path.join(original_dir, f)
        for f in os.listdir(original_dir)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]
    generated_images = [
        os.path.join(generated_dir, f)
        for f in os.listdir(generated_dir)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]

    min_len = min(len(original_images), len(generated_images))


    # # Calculate LPIPS
    transform_fid = transforms.Compose([
        transforms.Resize(image_size),  # Use training image_size [[5]]
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Match training normalization [[5]]
    ])

    tensors_A = []
    tensors_B = []
    for i in range(min_len):
        try:
            img_A = Image.open(original_images[i]).convert("RGB")
            img_B = Image.open(generated_images[i]).convert("RGB")

            tensors_A.append(transform_fid(img_A).to(device))
            tensors_B.append(transform_fid(img_B).to(device))
        except Exception as e:
            print(f"Error loading LPIPS tensors: {e}")
            continue

    if tensors_A and tensors_B:
        tensors_A = torch.stack(tensors_A)
        tensors_B = torch.stack(tensors_B)
        loss_fn = lpips.LPIPS(net='alex').to(device)
        avg_lpips = loss_fn(tensors_A, tensors_B).mean().item()
    else:
        avg_lpips = np.nan

    # Print results
    print(f"FID: {fid_score:.2f}")
    print(f"Inception Score (Mean ± Std): {is_mean:.2f} ± {metrics['inception_score_std']:.2f}")
    print(f"Kernel Inception Distance (Mean ± Std): {kid_mean:.4f} ± {metrics['kernel_inception_distance_std']:.4f}")
    print(f"LPIPS: {avg_lpips:.4f}")

In [ ]:
# a_dt_pth
# b_dt_pth
# a_ge_pth
# b_ge_pth
# a_dt_imgs
# b_dt_imgs
# a_ge_imgs
# b_ge_imgs
# ge_imgs
# path
evaluate_gan_metrics_1_4(
    a_dt_pth,
    ge_pth_g
)

# ConvNext V2+Swin Classification

## install libs and import

In [ ]:
%pip install torchinfo
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import transforms, models
from PIL import Image
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix
import time
import timm
import csv
import random
import seaborn as sns
import matplotlib.pyplot as plt
from torchinfo import summary

## initial variables

In [ ]:

# افزودن تمام تصاوير گلبولهاي سفيد براي آموزش يا حتي تست
root = '/content/drive/MyDrive/Colab Notebooks/Projects_Files/'
root_db = root + 'Blood_Cell/Rabin_White_Blood_Cell/'
cls = root_db + 'Classification/'
c_tr = cls + 'Train/'
c_ts = cls + 'Test/'
seg = root_db  + 'Segmentation/Original/'
pr_mdl = root + 'timm/conv_next_v2/pre_train_models/convnextv2_tiny_1k_224_fcmae.pt'
# sv_mdl = root + 'timm/conv_next_v2/models/convnextv2.pt'
# csv_ls = root + 'timm/conv_next_v2/loss_res/training_log.csv'
# '/content/drive/MyDrive/Colab Notebooks/Projects_Files/Blood_Cell/Rabin_White_Blood_Cell/Segmentation/Original/Basophil/xxx.jpg']

# paths = [cls+'Train/', cls+'Test/', seg] # کل 15657
# paths = [dvrs] # دایورس متنوع 362
or_pths = [c_tr, c_ts] #seg تکراریه
or_imgs = []
sl = '/'
fltrs=['Basophil'+sl,'Eosinophil'+sl,'Lymphocyte'+sl,'Monocyte'+sl,'Neutrophil'+sl]
# fltrs=['Eosinophil'+sl]
for path in or_pths:
  i=0
  # fldrs = [f for f in os.listdir(path)]
  fldrs = fltrs
  for fldr in fldrs:
      # print(fldr)
      len_fldr = [f for f in os.listdir(path+fldr) if f.endswith((".jpg", ".png", ".jpeg"))]
      for _file in len_fldr:
        or_imgs.append(path + fldrs[i] + _file)
      i = i+1
    # print(len(len_fldr))
or_imgs.sort()
print('original paths: ',len(or_imgs),or_imgs[-10:-1])



# Original images
ln_mn = 795
basophil_paths = [path for path in or_imgs if 'Basophil' in path]
print('basophil: ',len(basophil_paths),basophil_paths[-10:-1])
random.shuffle(basophil_paths)
basophil_paths = basophil_paths[:ln_mn]
print('basophil after: ',len(basophil_paths),basophil_paths)

eosinophil_paths = [path for path in or_imgs if 'Eosinophil' in path]
print('eosinophil: ',len(eosinophil_paths),eosinophil_paths[-10:-1])
random.shuffle(eosinophil_paths)
eosinophil_paths = eosinophil_paths[:ln_mn]
print('eosinophil after: ',len(eosinophil_paths),eosinophil_paths)

lymphocyte_paths = [path for path in or_imgs if 'Lymphocyte' in path]
print('lymphocyte: ',len(lymphocyte_paths),lymphocyte_paths[-10:-1])
random.shuffle(lymphocyte_paths)
lymphocyte_paths = lymphocyte_paths[:ln_mn]
print('lymphocyte after: ',len(lymphocyte_paths),lymphocyte_paths)

monocyte_paths = [path for path in or_imgs if 'Monocyte' in path]
print('monocyte: ',len(monocyte_paths),monocyte_paths[-10:-1])
random.shuffle(monocyte_paths)
monocyte_paths = monocyte_paths[:ln_mn]
print('monocyte after: ',len(monocyte_paths),monocyte_paths)

neutrophil_paths = [path for path in or_imgs if 'Neutrophil' in path]
print('neutrophil: ',len(neutrophil_paths),neutrophil_paths[-10:-1])
random.shuffle(neutrophil_paths)
neutrophil_paths = neutrophil_paths[:ln_mn]
print('neutrophil after: ',len(neutrophil_paths),neutrophil_paths)

or_imgs_tr = basophil_paths + eosinophil_paths + lymphocyte_paths + monocyte_paths +  neutrophil_paths
print('or_imgs_tr: ',len(or_imgs_tr),or_imgs_tr)

# Generated GAN
g_rt = '/content/drive/.shortcut-targets-by-id/1-3MblmfPfguMrEIIxIDUE4L0RaHDqFpt/GeG/'
g_imgs = []
g_pths = [g_rt] #seg تکراریه
for path in g_pths:
  i=0
  # fldrs = [f for f in os.listdir(path)]
  fldrs = fltrs
  for fldr in fldrs:
      len_fldr = [f for f in os.listdir(path+fldr) if f.endswith((".jpg", ".png", ".jpeg"))]
      for _file in len_fldr:
        g_imgs.append(path + fldrs[i] + _file)
      i = i+1
    # print(len(len_fldr))
g_imgs.sort()
print('g_imgs: ',len(g_imgs),g_imgs)


# GAN images
basophil_paths = [path for path in g_imgs if 'Basophil' in path]
print('basophil gan: ',len(basophil_paths),basophil_paths[-10:-1])
random.shuffle(basophil_paths)
# basophil_paths = basophil_paths[:ln_mn]
print('basophil gan after: ',len(basophil_paths),basophil_paths)

eosinophil_paths = [path for path in g_imgs if 'Eosinophil' in path]
print('eosinophil gan: ',len(eosinophil_paths),eosinophil_paths[-10:-1])
random.shuffle(eosinophil_paths)
eosinophil_paths = eosinophil_paths[:ln_mn]
print('eosinophil gan after: ',len(eosinophil_paths),eosinophil_paths)

lymphocyte_paths = [path for path in g_imgs if 'Lymphocyte' in path]
print('lymphocyte gan: ',len(lymphocyte_paths),lymphocyte_paths[-10:-1])
random.shuffle(lymphocyte_paths)
lymphocyte_paths = lymphocyte_paths[:600]
print('lymphocyte gan after: ',len(lymphocyte_paths),lymphocyte_paths)

monocyte_paths = [path for path in g_imgs if 'Monocyte' in path]
print('monocyte gan: ',len(monocyte_paths),monocyte_paths[-10:-1])
random.shuffle(monocyte_paths)
# monocyte_paths = monocyte_paths[:ln_mn]
print('monocyte gan after: ',len(monocyte_paths),monocyte_paths)

neutrophil_paths = [path for path in g_imgs if 'Neutrophil' in path]
print('neutrophil gan: ',len(neutrophil_paths),neutrophil_paths[-10:-1])
random.shuffle(neutrophil_paths)
neutrophil_paths = neutrophil_paths[:600]
print('neutrophil gan after: ',len(neutrophil_paths),neutrophil_paths)

g_imgs_tr = basophil_paths + eosinophil_paths + lymphocyte_paths + monocyte_paths +  neutrophil_paths
print('g_imgs_tr: ',len(g_imgs_tr),g_imgs_tr)





sv_mdl = root + 'timm/conv_next_v2/models/c_15g.pt'  # c_g2 c_g c c_3g
csv_ls = root + 'timm/conv_next_v2/loss_res/c_15g.csv'
print(sv_mdl)
print(csv_ls)

gln = len(g_imgs_tr)
print('gln: ',gln)
random.shuffle(g_imgs_tr)
g_imgs_tr = g_imgs_tr[:gln//15]
print('gln: ',gln)
image_paths = or_imgs_tr + g_imgs_tr
#or_imgs_tr + g_imgs_tr: 7046, or_imgs_tr(3565) + g_imgs_tr//2 : 5263, or_imgs_tr + g_imgs_tr//3: 4669   or_imgs_tr + g_imgs_tr//15 : 3718   or_imgs_tr (3565)



print('image_paths: ',len(image_paths),image_paths)

## extract labels from paths

In [ ]:
# استخراج برچسب از مسیر تصاویر
def get_label_from_path(path):
    return os.path.basename(os.path.dirname(path))  # مثال: 'x' یا 'y' [[1]]

labels = [get_label_from_path(p) for p in image_paths]
print(len(labels))
class_names = list(sorted(set(labels)))  # لیست کلاس ها
print(class_names)
num_classes = len(class_names)
print(num_classes)
# تبدیل برچسب به عدد
label_to_idx = {name: idx for idx, name in enumerate(class_names)}
print(len(label_to_idx))
print(label_to_idx)
targets = [label_to_idx[label] for label in labels]
print(targets)
print(len(targets))

## split data to train test validation

In [ ]:
# تقسیم داده به ترین، ولیدیشن، تست
current_seed = int(time.time())  # استفاده از زمان فعلی [[1]][[2]]

train_paths, temp_paths, train_targets, temp_targets = train_test_split(
    image_paths, targets, test_size=0.1, random_state=current_seed, shuffle=True
)
print(len(train_paths))
print(train_paths[0:5])
print(len(train_targets))
print(train_targets[0:5])
val_paths, test_paths, val_targets, test_targets = train_test_split(
    temp_paths, temp_targets, test_size=0.5, random_state=current_seed, shuffle=True
)

print(len(val_paths))
print(val_targets[0:5])
print(len(test_paths))
print(test_targets[0:5])

## generate dataset

In [ ]:
# کلاس دیتاست سفارشی
class CustomImageDataset(data.Dataset):
    def __init__(self, paths, targets, transform=None):
        self.paths = paths
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        label = self.targets[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label

## transformer and traing varibles

In [ ]:
epchs = 30
lr = 0.0002
best_val_loss = float('inf')
save_path = sv_mdl
im_sz = 256
t_im_sz = 224
rotate = 10
bch = 4
# تبدیل کننده تصاویر
train_transform = transforms.Compose([
    transforms.Resize(im_sz),  # تنظیم ابعاد [[5]][[7]]
    transforms.RandomHorizontalFlip(),  # افزونی داده [[5]]
    transforms.RandomRotation(rotate),      # دوران تصادفی [[5]]
    transforms.CenterCrop(t_im_sz),         # قطعه گیری مرکزی [[2]]
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize(im_sz),
    transforms.CenterCrop(t_im_sz),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# ساخت دیتاستها و لودرها
train_dataset = CustomImageDataset(train_paths, train_targets, transform=train_transform)
val_dataset = CustomImageDataset(val_paths, val_targets, transform=val_test_transform)
test_dataset = CustomImageDataset(test_paths, test_targets, transform=val_test_transform)

# train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = data.DataLoader(val_dataset, batch_size=32, shuffle=False)
# test_loader = data.DataLoader(test_dataset, batch_size=32, shuffle=False)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bch, shuffle=True, pin_memory=True, num_workers=4)  # [[2]][[4]]
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=bch, shuffle=False, pin_memory=True, num_workers=4)
test_loader = data.DataLoader(test_dataset, batch_size=bch, shuffle=False, pin_memory=True, num_workers=4)

## load ConvNext V2 model

In [ ]:

'''
۱. convnext_v2_tiny

    پارامترها : ~۵.۲ میلیون
    سرعت : بالا
    دقت : پایین تر از وariantهای بزرگتر
    استفاده : داده های کوچک و سیستم های کم منابع

    .


۲. convnext_v2_small

    پارامترها : ~۲۲ میلیون
    سرعت : متوسط
    دقت : مناسب برای کاربردهای عمومی
    استفاده : کلاسیفیکیشن تصاویر ساده

    .


۳. convnext_v2_base

    پارامترها : ~50 میلیون
    سرعت : مناسب
    دقت : بالا برای داده های متوسط

    .


۴. convnext_v2_large

    پارامترها : ~۱۳۴ میلیون
    سرعت : کندتر از variantهای کوچکتر
    دقت : بالاترین عملکرد

    .


۵. convnext_v2_huge

    پارامترها : ~۲۰۰ میلیون
    سرعت : بسیار کند
    دقت : بهینه برای داده های پیچیده

    .


'''

# ساخت مدل
# model = timm.create_model('hf_hub:timm/convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=True)  # [[1]][[2]]



# لود فایل وزن ها
checkpoint = torch.load(pr_mdl)
state_dict = checkpoint['model'] if 'model' in checkpoint else checkpoint  # حذف کلید 'model' [[2]]

# ایجاد مدل با timm
model = timm.create_model('convnextv2_tiny.fcmae', pretrained=False, num_classes=num_classes)  # [[1]]

# تنظیم state_dict با strict=False
model.load_state_dict(state_dict, strict=False)  # [[3]]



# حتما ایبجا رو ویژوال کن

summary(
    model,
    input_size=(1, 3, 224, 224),  # (batch, channels, height, width)
    col_names=["input_size", "output_size", "num_params", "kernel_size"],
    depth=1  # Increase to see more details
)

## implemant training

In [ ]:


# تنظیمات آموزش
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()  # تابع هزینه [[1]]
optimizer = optim.Adam(model.parameters(), lr=lr)




# ایجاد فایل CSV قبل از آموزش
csv_file = csv_ls
if not os.path.exists(csv_file):
    with open(csv_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Epoch', 'Train Loss (%)', 'Val Loss (%)'])

# حلقه آموزش
for epoch in range(epchs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        # images, labels = images.to(device), labels.to(device)
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)  # [[1]][[5]]
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)

    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    avg_train_loss = (train_loss / len(train_dataset)) * 100  # به درصد تبدیل [[1]]
    avg_val_loss = (val_loss / len(val_dataset)) * 100
    print(f'Epoch {epoch+1}/{epchs}, Train Loss: {avg_train_loss:.2f}%, Val Loss: {avg_val_loss:.2f}%')

    # ذخیره در فایل CSV
    with open(csv_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([epoch+1, avg_train_loss, avg_val_loss])

    # ذخیره مدل بهترین
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)
        print(f'Best model saved at {save_path}')

## Loading model for showing results as confusion matrix

In [ ]:

sv_mdl = root + 'timm/conv_next_v2/models/c_g.pt'  # c_g2 c_g c c_3g c_15g

save_path = sv_mdl

# {'Basophil': 0, 'Eosinophil': 1, 'Lymphocyte': 2, 'Monocyte': 3, 'Neutrophil': 4}
# Example class names (replace with your actual class names)
class_names = ['B', 'E', 'L', 'M', 'N']  # << Replace with your actual class names
themes = {
    'Blues': 'Blues',
    'Greens': 'Greens',
    'Reds': 'Reds',
    'Purples': 'Purples',
    'YlGnBu': 'YlGnBu',
    'YlOrRd': 'YlOrRd',
    'GnBu': 'GnBu',
    'BuPu': 'BuPu',
    'YlGn': 'YlGn',
    'PuBu': 'PuBu',
    'RdPu': 'RdPu',
    'PuRd': 'PuRd',
    'Blues_r': 'Blues_r',  # Reverse gradient
    'Coolwarm': 'coolwarm',  # Diverging colors
    'Seismic': 'seismic',    # High contrast
    'Viridis': 'viridis',    # Perceptually uniform
    'Plasma': 'plasma',
    'Inferno': 'inferno',
    'Magma': 'magma'
}

selected_theme = 'Greens'  # Change this to any of the themes above

fsz = 16
plt.rcParams.update({'font.size': fsz})  # Set base font size

# آزمون مدل
model = model.to(device)
model.load_state_dict(torch.load(save_path, map_location=device))
model.eval()

all_test_preds = []
all_test_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_test_preds.extend(preds)
        all_test_labels.extend(labels.cpu().numpy())

# محاسبه معیارهای ارزیابی
accuracy = accuracy_score(all_test_labels, all_test_preds)
f1 = f1_score(all_test_labels, all_test_preds,average='weighted') #,
recall = recall_score(all_test_labels, all_test_preds,average='weighted')

print(f'Test Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Recall: {recall:.4f}')

# محاسبه ماتریس اشتباه
cm = confusion_matrix(all_test_labels, all_test_preds)




# نمایش ماتریس اشتباه با نام کلاس و تم دلخواه
plt.figure(figsize=(12, 10),dpi=300)
sns.heatmap(cm,
            annot=True,
            fmt='d',
            cmap=themes[selected_theme],
            xticklabels=class_names,
            yticklabels=class_names,
            annot_kws={"size": fsz+10})  # Increase annotation text size

plt.title('Confusion Matrix of ConvNextV2 + SuperGAN', fontsize=fsz+10)  # Larger title fontweight='bold'
plt.xlabel('Predicted Class', fontsize=fsz+10)  # Larger label
plt.ylabel('Actual Class', fontsize=fsz+10)    # Larger label

plt.xticks(fontsize=fsz+10,rotation=0)  # Increase tick label size
plt.yticks(fontsize=fsz+10,rotation=0)

plt.tight_layout()  # Prevent label cutoff
# plt.savefig(os.path.join(save_dir, f'confusion_matrix_{selected_theme}.png'), dpi=300, bbox_inches='tight')
plt.show()
